# Курс «Алгоритмы анализа данных»

## Урок 6. Градиентный бустинг (AdaBoost)

### Домашняя работа к уроку 5

#### Задание 1

Для реализованной в методичке модели градиентного бустинга построить графики зависимости ошибки от количества деревьев в ансамбле и от максимальной глубины деревьев. 
Сделать выводы о зависимости ошибки от этих параметров.

В работе